In [1]:
import numpy
from sympy import *
from sympy.matrices import *

from generate_cpp_code import *
from utils import norm, jacobian

In [2]:
# Function("F")(MatrixSymbol("A", 2, 2)).diff()

In [3]:
generators = []

## Point-Edge

In [4]:
for dim in [2, 3]:
    p = numpy.array(symbols(" ".join("p_x p_y p_z".split()[:dim]))).reshape(dim, 1)
    e0 = numpy.array(symbols(" ".join("e0_x e0_y e0_z".split()[:dim]))).reshape(dim, 1)
    e1 = numpy.array(symbols(" ".join("e1_x e1_y e1_z".split()[:dim]))).reshape(dim, 1)
    x = numpy.vstack([p, e0, e1]).reshape(-1)
        
    e = e1 - e0
    c = ((p - e0).T @ e) / (e.T @ e)

    generators.append(CXXJacobianGenerator(c, x, f"point_edge_closest_point_{dim}D_jacobian"))

## Edge-Edge

In [5]:
ea0 = numpy.array(symbols("ea0_x ea0_y ea0_z"))
ea1 = numpy.array(symbols("ea1_x ea1_y ea1_z"))
eb0 = numpy.array(symbols("eb0_x eb0_y eb0_z"))
eb1 = numpy.array(symbols("eb1_x eb1_y eb1_z"))
x = numpy.vstack([ea0, ea1, eb0, eb1]).reshape(-1)

In [6]:
eb_to_ea = ea0 - eb0
ea = ea1 - ea0
eb = eb1 - eb0

coefMtr = numpy.array([
    [ea.dot(ea), -eb.dot(ea)],
    [-eb.dot(ea), eb.dot(eb)]
])

rhs = numpy.array([[-eb_to_ea.dot(ea)], [eb_to_ea.dot(eb)]])
c = Matrix(coefMtr).inv() @ rhs

In [7]:
generators.append(CXXJacobianGenerator(c, x, "edge_edge_closest_point_jacobian"))

## Point-Triangle

In [8]:
p = numpy.array(symbols("p_x p_y p_z")).reshape(3, 1)
t0 = numpy.array(symbols("t0_x t0_y t0_z")).reshape(3, 1)
t1 = numpy.array(symbols("t1_x t1_y t1_z")).reshape(3, 1)
t2 = numpy.array(symbols("t2_x t2_y t2_z")).reshape(3, 1)
x = numpy.vstack([p, t0, t1, t2]).reshape(-1)

In [9]:
basis = numpy.empty((2, 3), dtype=object)
basis[0] = (t1 - t0).T
basis[1] = (t2 - t0).T
A = basis @ basis.T
b = basis @ (p - t0)
display(Matrix(A))
display(Matrix(b))

Matrix([
[                                    (-t0_x + t1_x)**2 + (-t0_y + t1_y)**2 + (-t0_z + t1_z)**2, (-t0_x + t1_x)*(-t0_x + t2_x) + (-t0_y + t1_y)*(-t0_y + t2_y) + (-t0_z + t1_z)*(-t0_z + t2_z)],
[(-t0_x + t1_x)*(-t0_x + t2_x) + (-t0_y + t1_y)*(-t0_y + t2_y) + (-t0_z + t1_z)*(-t0_z + t2_z),                                     (-t0_x + t2_x)**2 + (-t0_y + t2_y)**2 + (-t0_z + t2_z)**2]])

Matrix([
[(p_x - t0_x)*(-t0_x + t1_x) + (p_y - t0_y)*(-t0_y + t1_y) + (p_z - t0_z)*(-t0_z + t1_z)],
[(p_x - t0_x)*(-t0_x + t2_x) + (p_y - t0_y)*(-t0_y + t2_y) + (p_z - t0_z)*(-t0_z + t2_z)]])

In [10]:
# f.write(gen_jac_code(numpy.array(Matrix(A).diff(x)).reshape(24, 2), x, 
#                      "point_triangle_LSQ_matrix_jacobian"))
# f.write(gen_jac_code(numpy.array(Matrix(b).diff(x)).reshape(12, 2).T, x, 
#                      "point_triangle_LSQ_RHS_jacobian"))

In [11]:
c = Matrix(A).inv() @ b

In [12]:
generators.append(CXXJacobianGenerator(c, x, "point_triangle_closest_point_jacobian"))

# Generate Code

In [13]:
generate_hpp_file(generators, "closest_point_grad.hpp")
generate_cpp_file(generators, "closest_point_grad.cpp")